<a href="https://colab.research.google.com/github/laq6661/Monocular-camera-project/blob/master/3d_sk_detect2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import imutils
import cv2
import os
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation

from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from google.colab import drive

Using TensorFlow backend.


In [2]:
labels=np.load('/content/drive/My Drive/data_sk/sk_label_h.npy')
print(labels[0:5])
print(np.shape(labels))

data = np.load('/content/drive/My Drive/data_sk/sk_data_h.npy')
#data = data.reshape(-1,50,3,1)
print(np.shape(data))


[32 32 32 32 32]
(1000000,)
(1000000, 50, 3)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data = np.array(data)
labels = np.array(labels)
# for a in range(1,53,1):
#   print("number of ",a,"data is:",list(labels).count(a))

In [4]:
le = LabelEncoder().fit(labels)   # LabelEncoder可以将标签分配一个0—n_classes-1之间的编码 
print(labels)
labels = np_utils.to_categorical(le.transform(labels), 51)  # 2是num_class表示输出的是2列数据的意思
print(np.shape(labels))

[32 32 32 ... 30 30 30]
(1000000, 51)


In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.15, 
                                                 stratify = labels, random_state = 42)

In [6]:
print(trainX.ndim)
print(trainX[0,:])
print(np.shape(trainY))
print(trainY)
print(np.shape(testX))
print(testY)

3
[[-0.144674 -0.411363  3.097303]
 [-0.139651 -0.123362  3.125123]
 [-0.133946  0.158757  3.141343]
 [-0.131911  0.299104  3.133884]
 [-0.292311  0.045759  3.115519]
 [-0.332721 -0.187986  3.084459]
 [-0.331357 -0.380837  2.997894]
 [-0.325683 -0.410171  2.996526]
 [ 0.018486  0.033946  3.133054]
 [ 0.045021 -0.211031  3.103108]
 [ 0.047422 -0.401408  3.012823]
 [ 0.035684 -0.430261  3.013935]
 [-0.211596 -0.403509  3.055614]
 [-0.221383 -0.718029  3.085544]
 [-0.188028 -1.016188  3.091671]
 [-0.178391 -1.08881   3.008924]
 [-0.074427 -0.40994   3.069063]
 [-0.085888 -0.743059  3.103921]
 [-0.10301  -1.030768  3.094378]
 [-0.12378  -1.089696  3.006313]
 [-0.13546   0.088921  3.139582]
 [-0.326866 -0.449089  2.988758]
 [-0.291622 -0.423978  2.978062]
 [ 0.043227 -0.473856  3.007998]
 [ 0.063672 -0.434276  2.994656]
 [ 0.        0.        0.      ]
 [ 0.        0.        0.      ]
 [ 0.        0.        0.      ]
 [ 0.        0.        0.      ]
 [ 0.        0.        0.      ]
 [ 0.   

In [7]:
model = Sequential()
model.add(Conv1D(input_shape=(50,3),kernel_size=3,filters=25,activation='relu',padding="same"))
model.add(MaxPooling1D(pool_size=3,strides=1,padding='same'))
model.add(Conv1D(kernel_size=3, filters=50,  activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(1024))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.2))
# model.add(Dense(1024))
# # #model.add(BatchNormalization())
# model.add(Activation('relu'))
model.add(Dense(51, activation='softmax'))
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 50, 25)            250       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 25)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 50)            3800      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 50)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1281024   


In [22]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

H = model.fit(trainX, trainY, validation_data = (testX, testY),
              batch_size = 128, epochs = 36, verbose = 1)  #verbose = 1显示进度条

Train on 850000 samples, validate on 150000 samples
Epoch 1/12
850000/850000 [==============================] - 72s 85us/step - loss: 0.0094 - acc: 0.9967 - val_loss: 0.0075 - val_acc: 0.9974
Epoch 2/12
850000/850000 [==============================] - 72s 84us/step - loss: 0.0093 - acc: 0.9967 - val_loss: 0.0078 - val_acc: 0.9974
Epoch 3/12
850000/850000 [==============================] - 71s 84us/step - loss: 0.0093 - acc: 0.9967 - val_loss: 0.0073 - val_acc: 0.9975
Epoch 4/12
850000/850000 [==============================] - 70s 82us/step - loss: 0.0092 - acc: 0.9968 - val_loss: 0.0074 - val_acc: 0.9975
Epoch 5/12
850000/850000 [==============================] - 75s 89us/step - loss: 0.0091 - acc: 0.9968 - val_loss: 0.0078 - val_acc: 0.9973
Epoch 6/12
850000/850000 [==============================] - 78s 91us/step - loss: 0.0091 - acc: 0.9968 - val_loss: 0.0074 - val_acc: 0.9975
Epoch 7/12
850000/850000 [==============================] - 77s 90us/step - loss: 0.0091 - acc: 0.9968 - val

In [23]:
predictions = model.predict(testX, batch_size = 64)
print(np.shape(predictions))
print(predictions.argmax(axis =1))
print(testY.argmax(axis = 1))
leclass = [str(i) for i in le.classes_]
print(leclass)
print(classification_report(testY.argmax(axis = 1), predictions.argmax(axis = 1),
                           target_names = leclass))

(150000, 51)
[49 29  4 ... 46 14 41]
[30 29  4 ... 46 14 41]
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51']
              precision    recall  f1-score   support

           1       0.95      0.94      0.94      1536
           2       0.94      0.95      0.95      5556
           3       0.95      0.95      0.95      4976
           4       0.96      0.92      0.94      2848
           5       0.94      0.92      0.93      2469
           6       0.87      0.88      0.88      2918
           7       0.95      0.93      0.94      2525
           8       0.96      0.95      0.95      4403
           9       0.96      0.95      0.95      1151
          10       0.96      0.98      0.97      5481
          11       0.96      0.97      0.96   

In [12]:
scores = model.evaluate(testX, testY, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 99.47%


In [0]:
model.save("/content/drive/My Drive/data_sk/sk_conv_93.hdf5")

In [0]:
from keras.models import load_model
loaded_model = load_model("/content/drive/My Drive/data_sk/sk_con1d4.hdf5")